In [ ]:
!pip install pandas numpy seaborn  datasets

### Installation des bibliothèques nécessaires (à exécuter une seule fois)


#### Importation des bibliothèques nécessaires


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datasets import load_dataset
import os 

In [1]:
import os 

In [6]:
chemin = 'data'
if not os.path.exists(chemin):
    os.makedirs(chemin)

In [ ]:
dataset = load_dataset("SaladSlayer00/ehr_nigeria_cancer")
train_set = dataset['train']
validation_set = dataset['validation']
test_set = dataset['test']

In [ ]:
train_set.to_csv('data/train.csv', index=False)
validation_set.to_csv('data/validation.csv', index=False)
test_set.to_csv('data/test.csv', index=False)

In [ ]:

# Chargement des données à partir d'un fichier CSV
df = pd.read_csv("data/train.csv")

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe

In [ ]:


# 1.1 Consistance des formats — Dates
# Conversion des colonnes de dates en format datetime pour une manipulation plus facile
df['Date_of_incidence'] = pd.to_datetime(df['Date_of_incidence'], errors='coerce')
df['Date_last_checked'] = pd.to_datetime(df['Date_last_checked'], errors='coerce')


In [ ]:

# 1.2 Consistance des formats — Textes
# Liste des colonnes textuelles à nettoyer
text_cols = ['Religion', 'Status', 'Marital_status', 'Education', 'Diagnosis', 'Treatment']

# Nettoyage des colonnes textuelles : suppression des espaces et conversion en minuscules
for col in text_cols:
    df[col] = df[col].astype(str).str.strip().str.lower()



    

In [ ]:


# 1.3 Présence d’outliers — Âge
# Identification des valeurs d'âge aberrantes (négatives ou supérieures à 120 ans)
print("Valeurs d'âge aberrantes :")
print(df[(df['Age'] < 0) | (df['Age'] > 120)][['Age']])


In [ ]:

# 1.4 Dates incohérentes : suivi avant l’incidence
# Calcul de la différence en jours entre Date_last_checked et Date_of_incidence
df['date_diff'] = (df['Date_last_checked'] - df['Date_of_incidence']).dt.days


# Identification des lignes où Date_last_checked est antérieure à Date_of_incidence
print("\nLignes où Date_last_checked < Date_of_incidence :")
print(df[df['date_diff'] < 0][['Date_of_incidence', 'Date_last_checked', 'date_diff']])


In [ ]:

# 1.5 Dead mais Date_last_checked très éloignée dans le futur (ex: +3 ans)
# Création d'un masque pour les patients décédés
dead_mask = df['Status'] == 'dead'
# Conversion de la différence en jours en années
df['years_diff'] = df['date_diff'] / 365.25

In [ ]:

# Identification des patients décédés avec une date de suivi supérieure à 3 ans
print("\nPatients morts avec date de suivi > 3 ans :")
print(df[dead_mask & (df['years_diff'] > 3)][['Date_of_incidence', 'Date_last_checked', 'years_diff']])



In [ ]:
# 1.6 Cas où 'Treatment' est 'unknown' mais 'Diagnosis' est rempli
# Identification des cas où le traitement est 'unknown' mais un diagnostic est connu
print("\nCas avec traitement 'unknown' mais un diagnostic connu :")
print(df[(df['Treatment'] == 'unknown') & (df['Diagnosis'].notnull()) & (df['Diagnosis'] != 'unknown')][['Diagnosis', 'Treatment']])


In [ ]:


# Conversion des colonnes de dates en format datetime pour une manipulation plus facile
df['Date_of_incidence'] = pd.to_datetime(df['Date_of_incidence'], errors='coerce')
df['Date_last_checked'] = pd.to_datetime(df['Date_last_checked'], errors='coerce')




In [ ]:

# 2.1 Âge
# Résumé statistique de l'âge
print("Résumé statistique de l'âge :")
print(df['Age'].describe())

# Visualisation de la distribution de l'âge
plt.figure(figsize=(10,4))
sns.histplot(df['Age'], kde=True, bins=30, color='skyblue')
plt.title("Distribution de l'âge")
plt.xlabel("Âge")
plt.show()

In [ ]:

# 2.2 Tribu
# Fréquence des tribus
print("\nFréquence des tribus :")
print(df['Tribe'].value_counts())

# Visualisation de la répartition par tribu
sns.countplot(data=df, y='Tribe', order=df['Tribe'].value_counts().index)
plt.title("Répartition par tribu")
plt.show()


In [ ]:


# 2.3 Statut vital
# Proportion vivant / décédé
print("\nProportion vivant / décédé :")
print(df['Status'].value_counts(normalize=True))

# Visualisation du statut vital
sns.countplot(data=df, x='Status')
plt.title("Statut vital")
plt.show()

In [ ]:


# 2.4 Traitement
# Types de traitements
print("\nTypes de traitements :")
print(df['Treatment'].value_counts())

# Visualisation des types de traitement
sns.countplot(data=df, y='Treatment', order=df['Treatment'].value_counts().index)
plt.title("Types de traitement")
plt.show()

In [ ]:


# 2.5 Diagnostic
# Types de diagnostic
print("\nTypes de diagnostic :")
print(df['Diagnosis'].value_counts())

# Visualisation des modes de diagnostic
sns.countplot(data=df, y='Diagnosis', order=df['Diagnosis'].value_counts().index)
plt.title("Modes de diagnostic")
plt.show()


In [ ]:
# 2.6 Morphologie tumorale
# Types de morphologie tumorale (top 10)
print("\nTypes de morphologie tumorale :")
print(df['Morphology'].value_counts().head(10))

# Visualisation de la morphologie tumorale (top 10)
plt.figure(figsize=(10,6))
sns.countplot(data=df, y='Morphology', order=df['Morphology'].value_counts().iloc[:10].index)
plt.title("Morphologie tumorale (top 10)")
plt.show()


In [ ]:


# 2.7 Quadrant du sein affecté
# Quadrants affectés
print("\nQuadrants affectés :")
print(df['Breast_quadrant'].value_counts())

# Visualisation du quadrant mammaire affecté
sns.countplot(data=df, y='Breast_quadrant', order=df['Breast_quadrant'].value_counts().index)
plt.title("Quadrant mammaire affecté")
plt.show()


In [ ]:

# 2.8 Différenciation tumorale
# Niveaux de différenciation tumorale
print("\nNiveaux de différenciation tumorale :")
print(df['Tumour_differentiation'].value_counts())

# Visualisation de la différenciation tumorale
sns.countplot(data=df, x='Tumour_differentiation', order=df['Tumour_differentiation'].value_counts().index)
plt.title("Différenciation tumorale")
plt.show()

# 2.9 Religion et état matrimonial
# Distribution de la religion
print("\nDistribution de la religion :")
print(df['Religion'].value_counts())


In [ ]:

# Religion
plt.subplot(2, 3, 6)
df['Religion'].value_counts().plot(kind='bar')
plt.title('Répartition religieuse')
plt.show()

In [ ]:
# Tribe
plt.subplot(2, 3, 5)
df['Tribe'].value_counts().plot(kind='bar')
plt.title('Répartition par tribu')
plt.show()

In [ ]:
df.columns 


### Création de figures séparées pour chaque analyse bivariée


In [ ]:


# 1. Âge vs Statut vital
# Visualisation de la distribution de l'âge en fonction du statut vital
plt.figure(figsize=(6, 4))
sns.boxplot(x='Status', y='Age', data=df)
plt.title('Âge vs Statut vital')
plt.show()


In [ ]:

# 2. Traitement vs Statut vital
# Visualisation de la relation entre le traitement et le statut vital
plt.figure(figsize=(6, 4))
sns.countplot(x='Treatment', hue='Status', data=df)
plt.title('Traitement vs Statut vital')
plt.xticks(rotation=45)
plt.show()

In [ ]:

# 3. Tribu vs Traitement
# Visualisation de la relation entre la tribu et le traitement
plt.figure(figsize=(6, 4))
sns.countplot(x='Tribe', hue='Treatment', data=df)
plt.title('Tribu vs Traitement')
plt.xticks(rotation=45)
plt.show()

In [ ]:


# 4. Tribu vs Diagnostic
# Visualisation de la relation entre la tribu et le diagnostic
plt.figure(figsize=(6, 4))
sns.countplot(x='Tribe', hue='Diagnosis', data=df)
plt.title('Tribu vs Diagnostic')
plt.xticks(rotation=45)
plt.show()

In [ ]:

# 6. Éducation vs Type de traitement
# Visualisation de la relation entre l'éducation et le type de traitement
plt.figure(figsize=(6, 4))
sns.countplot(x='Education', hue='Treatment', data=df)
plt.title('Éducation vs Traitement')
plt.xticks(rotation=45)
plt.show()

In [ ]:


# 7. Religion vs Type de traitement
# Visualisation de la relation entre la religion et le type de traitement
plt.figure(figsize=(6, 4))
sns.countplot(x='Religion', hue='Treatment', data=df)
plt.title('Religion vs Traitement')
plt.xticks(rotation=45)
plt.show()


In [ ]:

# 8. Durée de suivi (en années) entre diagnostic et dernier contrôle
# Visualisation de la distribution de la durée de suivi en années
plt.figure(figsize=(6, 4))
sns.histplot(df['years_diff'].dropna(), kde=True)
plt.title('Durée de suivi (en années)')
plt.xlabel('Années entre diagnostic et dernier contrôle')
plt.show()

In [ ]:




# 9. Statut marital vs Statut vital
# Visualisation de la relation entre le statut marital et le statut vital
plt.figure(figsize=(6, 4))
sns.countplot(x='Marital_status', hue='Status', data=df)
plt.title('Statut marital vs Statut vital')
plt.xticks(rotation=45)
plt.show()


In [ ]:

# 5. Différenciation tumorale vs Morphologie
plt.figure(figsize=(20, 14))
sns.countplot(x='Tumour_differentiation', hue='Morphology', data=df)
plt.title('Différenciation tumorale vs Morphologie')
plt.xticks(rotation=45)
plt.show()


In [ ]:

# Créer une colonne d'année d'incidence
# Extraction de l'année à partir de la date d'incidence
df['Incidence_year'] = df['Date_of_incidence'].dt.year

In [ ]:


# Créer une colonne d'année de dernier suivi
# Extraction de l'année à partir de la date du dernier suivi
df['Last_check_year'] = df['Date_last_checked'].dt.year


In [ ]:

# Créer une colonne de durée en jours et en années si ce n'est pas déjà fait
# Calcul de la durée en jours entre la date d'incidence et la date du dernier suivi
df['duration_days'] = (df['Date_last_checked'] - df['Date_of_incidence']).dt.days
# Conversion de la durée en années
df['duration_years'] = df['duration_days'] / 365.25


In [ ]:

# Évolution dans le temps : nombre de cas par année
# Visualisation du nombre de cas par année de diagnostic
plt.figure(figsize=(10, 5))
sns.countplot(data=df, x='Incidence_year', order=sorted(df['Incidence_year'].dropna().unique()))
plt.title('Nombre de cas par année de diagnostic')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:

# Durée entre diagnostic et dernier suivi (distribuée)
# Visualisation de la distribution de la durée entre le diagnostic et le dernier suivi en années
plt.figure(figsize=(10, 5))
sns.histplot(df['duration_years'].dropna(), bins=30, kde=True)
plt.title("Durée entre le diagnostic et le dernier suivi (en années)")
plt.xlabel("Années")
plt.tight_layout()
plt.show()

In [ ]:


# Mortalité par année (proportion de 'Dead' par année)
# Calcul de la proportion de décès par année d'incidence
mortality_by_year = df.groupby('Incidence_year')['Status'].value_counts(normalize=True).unstack().fillna(0)



In [ ]:
# Visualisation du taux de mortalité par année d'incidence
plt.figure(figsize=(10, 5))
mortality_by_year['dead'].plot(kind='bar', color='tomato', label='Taux de mortalité')
plt.ylabel("Proportion de décès")
plt.title("Taux de mortalité par année d'incidence")
plt.tight_layout()
plt.show()


In [ ]:
print(df['Status'].unique())

In [ ]:
df.columns

In [ ]:

plt.figure(figsize=(12, 6))
sns.countplot(data=df, x='Tribe', order=df['Tribe'].value_counts().index)
plt.title("Nombre de cas par tribu")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
# Standardiser la colonne Status
df['Status'] = df['Status'].str.strip().str.lower()

# Taux de mortalité par tribu
mortality_by_tribe = df.groupby('Tribe')['Status'].value_counts(normalize=True).unstack().fillna(0)

if 'dead' in mortality_by_tribe.columns:
    plt.figure(figsize=(12, 6))
    mortality_by_tribe['dead'].plot(kind='bar', color='darkred')
    plt.title("Taux de mortalité par tribu")
    plt.ylabel("Proportion de décès")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
else:
    print("La colonne 'dead' n'existe pas. Vérifie les valeurs de la colonne Status.")


In [ ]:
plt.figure(figsize=(14, 6))
sns.countplot(data=df, x='Tribe', hue='Treatment', order=df['Tribe'].value_counts().index)
plt.title("Traitements reçus par tribu")
plt.xlabel("Tribu")
plt.ylabel("Nombre de patientes")
plt.xticks(rotation=45)
plt.legend(title='Traitement')
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(14, 6))
sns.countplot(data=df, x='Tribe', hue='Diagnosis', order=df['Tribe'].value_counts().index)
plt.title("Diagnostic par tribu")
plt.xlabel("Tribu")
plt.ylabel("Nombre de patientes")
plt.xticks(rotation=45)
plt.legend(title='Diagnostic')
plt.tight_layout()
plt.show()


In [ ]:
df.columns

In [ ]:
# Statut en minuscules (normalisation)
df['Status'] = df['Status'].str.lower().str.strip()

# Distribution des statuts par tribu
status_by_tribe = df.groupby('Tribe')['Status'].value_counts(normalize=True).unstack().fillna(0)

# Affichage du taux de mortalité
status_by_tribe[['alive', 'dead']].plot(kind='bar', stacked=True, figsize=(12, 6), colormap='coolwarm')
plt.title("Distribution Alive vs Dead par tribu (équilibre des classes)")
plt.ylabel("Proportion")
plt.xlabel("Tribu")
plt.xticks(rotation=45)
plt.legend(title='Statut')
plt.tight_layout()
plt.show()


In [ ]:
sensitive_variables = ['Tribe', 'Religion', 'Education', 'Marital_status']
print("🔐 Variables sensibles détectées :")
for var in sensitive_variables:
    print(f" - {var}")


In [ ]:
age_by_tribe = df.groupby('Tribe')['Age'].mean().sort_values()
age_by_tribe.plot(kind='barh', figsize=(10, 6), color='skyblue')
plt.title("Âge moyen par tribu (variation inter-silos)")
plt.xlabel("Âge moyen")
plt.tight_layout()
plt.show()


In [ ]:
major_treatment = df.groupby('Tribe')['Treatment'].agg(lambda x: x.value_counts().index[0])
print("Traitement le plus fréquent par tribu :\n", major_treatment)
